In [159]:
current_player = True
def is_valid_move(board,start,end,castling_valid):
  global current_player
  original_board = board
  board = [row[:] for row in original_board]
  if not(start[0] in range(8) and start[1] in range(8) and end[0] in range(8) and end[1] in range(8)):
    return False,board
  start_role = board[start[0]][start[1]]
  end_role = board[end[0]][end[1]]
  if start_role == '.':
    return False,board
  if current_player == start_role.isupper():
    current_player = not current_player
  else:
    return False,board
  if end_role != '.' and start_role.isupper()==end_role.isupper():
    return False,board

  def move():
    board[end[0]][end[1]]=start_role
    board[start[0]][start[1]]="."

  def is_path_clear(start,end):
    if start[0]==end[0]:
      for i in range(min(start[1],end[1])+1,max(start[1],end[1])):
        if board[start[0]][i]!=".":
          return False
    elif start[1]==end[1]:
      for i in range(min(start[0],end[0])+1,max(start[0],end[0])):
        if board[i][start[1]]!=".":
          return False
    else:
      row_step = 1 if start[0]<end[0] else -1
      col_step = 1 if start[1]<end[1] else -1
      for i in range(1,abs(start[0]-end[0])):
        if board[start[0]+row_step*i][start[1]+col_step*i]!=".":
          return False
    return True


  def promote():
    while True:
      promotion = input("Promote pawn to (q, r, b, n): ").lower()
      if promotion in ['q', 'r', 'b', 'n']:
        break
      else:
        print("Invalid input. Please promote pawn to (q, r, b, n).")
    board[end[0]][end[1]]=promotion.upper() if board[end[0]][end[1]].isupper() else promotion.lower()



  def is_position_safe(pos):
    for row in range(8):
      for col in range(8):
        piece = board[row][col]
        if (piece!=".") and (piece.isupper()!=start_role.isupper()):
          if eatable([row, col], pos):
            return False
    return True

  def eatable(start, end):
    start_piece = board[start[0]][start[1]]
    if start_piece.lower() == 'p':
      direction = -1 if start_role.isupper() else 1
      if (end[0]==start[0]+direction and abs(end[1]-start[1])==1):
        return True
    if start_piece.lower() == 'r':
      if (start[0] == end[0] or start[1] == end[1]) and is_path_clear(start, end):
        return True
    if start_piece.lower() == 'b':
      if abs(start[0]-end[0])==abs(start[1]-end[1]) and is_path_clear(start, end):
        return True
    if start_piece.lower() == 'n':
      if abs(start[0]-end[0])==2 and abs(start[1]-end[1])==1 or abs(start[0]-end[0])==1 and abs(start[1]-end[1])==2:
        return True
    if start_piece.lower() == 'q':
      if (start[0] == end[0] or start[1] == end[1] or abs(start[0]-end[0])==abs(start[1]-end[1])) and is_path_clear(start, end):
        return True
    if start_piece.lower() == 'k':
      if abs(start[0]-end[0])<=1 and abs(start[1]-end[1])<=1:
        return True
    return False

  def find_king_pos():
    for row in range(8):
      for col in range(8):
        if board[row][col].lower() == "k" and board[row][col].isupper() == start_role.isupper():
          king_pos = [row, col]
          break
    return king_pos

  if start_role.lower()=="k" and abs(start[1]-end[1])==2:
    if castling_valid['lu'] and start == [0,4] and end == [0,2]:
      for i in range(1,4):
        if board[0][i]!=".":
          return False,board
      for i in range(2,5):
        if not is_position_safe([0,i]):
          return False,board
      board[0][2]="k"
      board[0][3]="r"
      board[0][0]="."
      board[0][4]="."
      castling_valid['lu']=False
      castling_valid['ru']=False
      return True,board
    if castling_valid['ru'] and start == [0,4] and end == [0,6]:
      for i in range(5,7):
        if board[0][i]!=".":
          return False,board
      for i in range(4,7):
        if not is_position_safe([0,i]):
          return False,board
      board[0][6]="k"
      board[0][5]="r"
      board[0][7]="."
      board[0][4]="."
      castling_valid['lu']=False
      castling_valid['ru']=False
      return True,board
    if castling_valid['lb'] and start == [7,4] and end == [7,2]:
      for i in range(1,4):
        if board[7][i]!=".":
          return False,board
      for i in range(2,5):
        if not is_position_safe([7,i]):
          return False,board
      board[7][2]="K"
      board[7][3]="R"
      board[7][0]="."
      board[7][4]="."
      castling_valid['lb']=False
      castling_valid['rb']=False
      return True,board
    if castling_valid['rb'] and start == [7,4] and end == [7,6]:
      for i in range(5,7):
        if board[7][i]!=".":
          return False,board
      for i in range(4,7):
        if not is_position_safe([7,i]):
          return False,board
      board[7][6]="K"
      board[7][5]="R"
      board[7][7]="."
      board[7][4]="."
      castling_valid['lb']=False
      castling_valid['rb']=False
      return True,board


  if start_role.lower()=="p":
    direction = -1 if start_role.isupper() else 1
    if (end[0]==start[0]+direction and start[1]==end[1] and board[end[0]][end[1]]==".") or\
    (start[0] in [1,6] and end[0]==start[0]+direction*2 and start[1]==end[1] and board[end[0]][end[1]]==".") or\
    (end[0]==start[0]+direction and abs(end[1]-start[1])==1 and board[end[0]][end[1]]!="."):
      move()

      if end[0]==start[0]+direction*2:
        if end[1]-1>=0 and board[end[1]-1][end[0]] == start_role.swapcase():
          while True:
            en_passant = input("Whether do left side en passant? Y/N").lower()
            if en_passant in ['y', 'n']:
              break
            else:
              print("Invalid input. Please enter Y or N.")
          if en_passant == 'y':
            board[end[0]][end[1]-1] = '.'
            board[end[0]][end[1]] = '.'
            board[end[0]-direction][end[1]] = start_role.swapcase()
            current_player = not current_player
            if not is_position_safe(find_king_pos()):
              board = [row[:] for row in original_board]
              return False,board
            return True,board

        if end[1]+1<8 and board[end[1]+1][end[0]] == start_role.swapcase():
          while True:
            en_passant = input("Whether do right side en passant? Y/N").lower()
            if en_passant in ['y', 'n']:
              break
            else:
              print("Invalid input. Please enter Y or N.")
          if en_passant == 'y':
            board[end[0]][end[1]+1] = '.'
            board[end[0]][end[1]] = '.'
            board[end[0]-direction][end[1]] = start_role.swapcase()
            current_player = not current_player
            if not is_position_safe(find_king_pos()):
              board = [row[:] for row in original_board]
              return False,board
            return True,board




      if end[0] in [7,0]:
        promote()
      return True,board
    else:
      return False,board

  if start_role.lower()=="r":
    if (start[0]==end[0] or start[1]==end[1]) and is_path_clear(start,end):
      move()
      if not is_position_safe(find_king_pos()):
        board = [row[:] for row in original_board]
        return False,board
      return True,board
    else:
      return False,board

  if start_role.lower()=="b":
    if abs(start[0]-end[0])==abs(start[1]-end[1]) and is_path_clear(start,end):
      if start[0]==0 and start[1]==0:
        castling_valid['lu']=False
      elif start[0]==0 and start[1]==7:
        castling_valid['ru']=False
      elif start[0]==7 and start[1]==0:
        castling_valid['lb']=False
      else:
        castling_valid['rb']=False
      move()
      if not is_position_safe(find_king_pos()):
        board = [row[:] for row in original_board]
        return False,board
      return True,board
    else:
      return False,board

  if start_role.lower()=="n":
    if abs(start[0]-end[0])==2 and abs(start[1]-end[1])==1 or abs(start[0]-end[0])==1 and abs(start[1]-end[1])==2:
      move()
      if not is_position_safe(find_king_pos()):
        board = [row[:] for row in original_board]
        return False,board
      return True,board
    else:
      return False,board

  if start_role.lower()=="q":
    if (start[0]==end[0] or start[1]==end[1] or abs(start[0]-end[0])==abs(start[1]-end[1])) and is_path_clear(start,end):
      move()
      if not is_position_safe(find_king_pos()):
        board = [row[:] for row in original_board]
        return False,board
      return True,board
    else:
      return False,board

  if start_role.lower()=="k":
    if abs(start[0]-end[0])<=1 and abs(start[1]-end[1])<=1 and is_position_safe(end):
      if start[0]==0:
        castling_valid['lu']=False
        castling_valid['ru']=False
      elif start[0]==7:
        castling_valid['lb']=False
        castling_valid['rb']=False
      move()
      return True,board
    else:
      return False,board

In [163]:
chess_board = [
    ['r', '.', 'b', 'q', 'k', 'b', 'n', 'r'],
    ['p', 'P', 'p', 'p', 'p', 'p', 'p', 'p'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
    ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
]
new_board = chess_board
castling_valid = {
    'lb': True,
    'rb': True,
    'lu': True,
    'ru': True
}
current_player = True #if you want lowcase player start first, turn it into False

In [164]:
start = [1,1]
end = [0,0]

is_valid,new_board = is_valid_move(new_board,start,end,castling_valid)
if not is_valid:
  print("invalid move")

for row in new_board:
  print(row)


Promote pawn to (q, r, b, n): q
['Q', '.', 'b', 'q', 'k', 'b', 'n', 'r']
['p', '.', 'p', 'p', 'p', 'p', 'p', 'p']
['.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P']
['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
